In [37]:
import os
import sys
import numpy as np
import torch

from datetime import datetime
from typing import Tuple
from torch import nn
from torch.nn import Module
import torch.nn.functional as F
from torch.nn import KLDivLoss, CrossEntropyLoss, CosineEmbeddingLoss, MSELoss
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

### Loading Teacher models

In [38]:
import clip

model_name = "ViT-B/32"

# model is the torch model.
# preprocess function is for image preprocessing.

model, preprocess = clip.load(model_name)

# Get only the visual model
visual_teacher_model = model.visual
text_teacher_model = model.transformer


input_resolution = model.visual.input_resolution

### Instantiating Student models

[VisionTransformer](https://github.com/openai/CLIP/blob/d50d76daa670286dd6cacf3bcd80b5e4823fc8e1/clip/model.py#L206)
[Transformer](https://github.com/openai/CLIP/blob/d50d76daa670286dd6cacf3bcd80b5e4823fc8e1/clip/model.py#L195)

In [39]:
from clip.model import Transformer, VisionTransformer
from clip.model import convert_weights  # Make them float16

# Set Student Configuration

patch_size = 32
width = 384
layers = 6
heads = 12
output_dim = 512

visual_student_model = VisionTransformer(
    input_resolution=input_resolution,
    patch_size=patch_size,
    width=width,
    layers=layers,
    heads=heads,
    output_dim=output_dim,
)

width = 512
layers = 6
heads = 8  # More Number of Heads


def build_attention_mask():
    context_length = 77
    mask = torch.empty(context_length, context_length)
    mask.fill_(float("-inf"))
    mask.triu_(1)  # zero out the lower diagonal
    return mask


text_student_model = Transformer(
    width=width, layers=layers, heads=heads, attn_mask=build_attention_mask()
)


convert_weights(visual_student_model)
convert_weights(text_student_model)

In [40]:
def encode_text(transformer, text):

    x = model.token_embedding(text).type(model.dtype)  # [batch_size, n_ctx, d_model]

    x = x + model.positional_embedding.type(model.dtype)
    x = x.permute(1, 0, 2)  # NLD -> LND

    x = transformer(x)

    x = x.permute(1, 0, 2)  # LND -> NLD
    x = model.ln_final(x).type(model.dtype)

    # x.shape = [batch_size, n_ctx, transformer.width]
    # take features from the eot embedding (eot_token is the highest number in each sequence)
    x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ model.text_projection

    return x

In [41]:
class ProjectionHead(nn.Module):
    def __init__(self, embedding_dim=512, projection_dim=512, dropout=0.2):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

### Load the WIT Dataset

In [42]:
# from concurrent.futures import ThreadPoolExecutor
# from functools import partial
# import io
# import urllib

# import PIL.Image

# from datasets import load_dataset
# from datasets.utils.file_utils import get_datasets_user_agent


# def fetch_single_image(image_url, timeout=None, retries=0):
#     for _ in range(retries + 1):
#         try:
#             request = urllib.request.Request(
#                 image_url,
#                 data=None,
#                 headers={"user-agent": get_datasets_user_agent()},
#             )
#             with urllib.request.urlopen(request, timeout=timeout) as req:
#                 image = PIL.Image.open(io.BytesIO(req.read()))
#             break
#         except Exception:
#             image = None
#     return image


# def fetch_images(batch, num_threads, timeout=None, retries=0):
#     fetch_single_image_with_args = partial(
#         fetch_single_image, timeout=timeout, retries=retries
#     )
#     with ThreadPoolExecutor(max_workers=num_threads) as executor:
#         batch["image"] = list(
#             executor.map(fetch_single_image_with_args, batch["image_url"])
#         )
#     return batch


# num_threads = 20
# dset = load_dataset("conceptual_captions",split='train[:10000]')
# # dset = dset.remove_columns("image_url")

# dset = dset.filter(lambda example: len(example["caption"]) < 75)

# dset = dset.map(
#     fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads}
# )

# dset = dset.filter(lambda example : example["image"] is not None)

# def transform_func(examples):
#     examples["image"] = [preprocess(img) for img in examples["image"]]
#     return examples

# dset = dset.with_transform(transform_func)

### Convert Dataset into Torch Dataloader

In [43]:
from torch.utils.data import DataLoader
from datasets import load_from_disk

dset = load_from_disk("./data/processed")


def transform_func(examples):
    examples["image"] = [preprocess(img) for img in examples["image"]]
    return examples


dset = dset.with_transform(transform_func)

train_dataloader = DataLoader(dset, batch_size=16, shuffle=True, num_workers=8)

In [44]:
dset

Dataset({
    features: ['caption', 'image'],
    num_rows: 7012
})

In [52]:
class NSTLoss(nn.Module):
    """
    Like What You Like: Knowledge Distill via Neuron Selectivity Transfer
    https://arxiv.org/pdf/1707.01219.pdf
    """

    def __init__(self):
        super(NSTLoss, self).__init__()

    def forward(self, fm_s, fm_t):
        fm_s = F.normalize(fm_s, dim=1)
        fm_t = F.normalize(fm_t, dim=1)

        loss = (
            self.poly_kernel(fm_s, fm_s).mean()
            - 2 * self.poly_kernel(fm_s, fm_t).mean()
        )

        return loss

    def poly_kernel(self, fm1, fm2):
        fm1 = fm1.unsqueeze(1)
        fm2 = fm2.unsqueeze(2)
        out = (fm1 * fm2).sum(-1).pow(2)

        return out

In [53]:
class DistillationTrainer:
    def __init__(self, *args, **kwargs):

        self.visual_teacher = visual_teacher_model
        self.text_teacher = text_teacher_model

        self.visual_student = visual_student_model
        self.text_student = text_student_model

        self.image_projection = ProjectionHead().half()
        self.text_projection = ProjectionHead().half()
        self.temperature = 1

        self.train_dataloader = train_dataloader

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.visual_teacher = self.visual_teacher.to(self.device)
        self.text_teacher = self.text_teacher.to(self.device)
        self.visual_student = self.visual_student.to(self.device)
        self.text_student = self.text_student.to(self.device)
        self.image_projection = self.image_projection.to(self.device)
        self.text_projection = self.text_projection.to(self.device)

        self.mmd_loss = NSTLoss()

        self.visual_teacher.eval()
        self.text_teacher.eval()

        self.epochs = 30
        self.start_epoch = 1

        # set up optimizer
        self.optimizer = SGD(
            list(self.visual_student.parameters())
            + list(self.text_student.parameters()),
            lr=0.001,
        )

    def compute_loss(self, images, texts, return_outputs=False):
        texts = clip.tokenize(texts)

        texts = texts.to(self.device)

        images = images.to(self.device).half()

        visual_outputs_student = self.visual_student(images)
        text_outputs_student = encode_text(self.text_student, texts)
        #         print(visual_outputs_student.size())
        #         print(text_outputs_student.size())

        # compute teacher output
        with torch.no_grad():
            visual_outputs_teacher = self.visual_teacher(images)
            text_outputs_teacher = model.encode_text(texts)

        #         print(visual_outputs_teacher.size())
        #         print(text_outputs_student.size())
        # assert size
        assert visual_outputs_student.size() == visual_outputs_teacher.size()
        assert text_outputs_student.size() == text_outputs_teacher.size()

        # Push visual_outputs_student and text_outputs_student closer

        image_embeddings = self.image_projection(visual_outputs_student)
        text_embeddings = self.text_projection(text_outputs_student)

        ## NST loss
        text_nst = 0
        image_nst = 0
        for i in range(image_embeddings.size(0)):
            text_nst += self.mmd_loss(
                text_embeddings[i : i + 1], text_outputs_teacher[i : i + 1]
            )
            image_nst += self.mmd_loss(
                image_embeddings[i : i + 1], visual_outputs_teacher[i : i + 1]
            )

        text_nst /= text_embeddings.size(1)
        image_nst /= image_embeddings.size(1)

        ## main loss

        logits = (text_embeddings @ image_embeddings.T) / self.temperature

        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T

        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )

        texts_loss = self.cross_entropy(logits, targets, reduction="none")
        images_loss = self.cross_entropy(logits.T, targets.T, reduction="none")
        con_loss = (images_loss + texts_loss) / 2.0
        con_loss = con_loss.mean()

        ## total loss
        loss = (text_nst + image_nst + con_loss) / 3

        return loss

    def cross_entropy(self, preds, targets, reduction="none"):
        log_softmax = nn.LogSoftmax(dim=-1)
        loss = (-targets * log_softmax(preds)).sum(1)
        if reduction == "none":
            return loss
        elif reduction == "mean":
            return loss.mean()

    def train(self):
        for epoch in range(self.start_epoch, self.epochs + 1):
            print(f"Starting Epoch {epoch} ------------------------------------------")
            loss_value = self._train_epoch(epoch)
            print(f"Combined Loss Value after {epoch} Epoch is {loss_value}")

    def _train_epoch(self, epoch):
        loss_value = 0
        for batch_idx, data in enumerate(self.train_dataloader):

            self.optimizer.zero_grad()

            texts = data["caption"]
            images = data["image"]

            loss = self.compute_loss(images, texts)

            loss_value += loss

            loss.backward()

            self.optimizer.step()

            if batch_idx % 100 == 0:
                print(
                    f"Loss after {batch_idx}/{len(self.train_dataloader)} Batch is {loss_value/(batch_idx+1)} "
                )

        return loss_value.detach().cpu().numpy() / len(self.train_dataloader)

In [54]:
Trainer = DistillationTrainer(
    visual_teacher_model=visual_teacher_model,
    text_teacher_model=text_teacher_model,
    text_student_model=text_student_model,
    visual_student_model=visual_student_model,
    train_dataloader=train_dataloader,
)

In [55]:
Trainer.train()

Starting Epoch 1 ------------------------------------------
Loss after 0/439 Batch is 5.515625 
Loss after 100/439 Batch is 3.244140625 
Loss after 200/439 Batch is 2.552734375 
Loss after 300/439 Batch is 2.2421875 
Loss after 400/439 Batch is 2.04296875 
Combined Loss Value after 1 Epoch is 1.9851936218678816
Starting Epoch 2 ------------------------------------------
Loss after 0/439 Batch is 1.3916015625 
Loss after 100/439 Batch is 1.3095703125 
Loss after 200/439 Batch is 1.283203125 
Loss after 300/439 Batch is 1.259765625 
Loss after 400/439 Batch is 1.244140625 
Combined Loss Value after 2 Epoch is 1.2323462414578588
Starting Epoch 3 ------------------------------------------
Loss after 0/439 Batch is 1.0859375 
Loss after 100/439 Batch is 1.1123046875 
Loss after 200/439 Batch is 1.1025390625 
Loss after 300/439 Batch is 1.0859375 
Loss after 400/439 Batch is 1.0771484375 
Combined Loss Value after 3 Epoch is 1.0734624145785876
Starting Epoch 4 -------------------------------

Loss after 200/439 Batch is 0.80224609375 
Loss after 300/439 Batch is 0.80615234375 
Loss after 400/439 Batch is 0.798828125 
Combined Loss Value after 26 Epoch is 0.7949886104783599
Starting Epoch 27 ------------------------------------------
Loss after 0/439 Batch is 0.67431640625 
Loss after 100/439 Batch is 0.79833984375 
Loss after 200/439 Batch is 0.7978515625 
Loss after 300/439 Batch is 0.79638671875 
Loss after 400/439 Batch is 0.79248046875 
Combined Loss Value after 27 Epoch is 0.7887243735763098
Starting Epoch 28 ------------------------------------------
Loss after 0/439 Batch is 0.93505859375 
Loss after 100/439 Batch is 0.78662109375 
Loss after 200/439 Batch is 0.7880859375 
Loss after 300/439 Batch is 0.7880859375 
Loss after 400/439 Batch is 0.78564453125 
Combined Loss Value after 28 Epoch is 0.7841685649202733
Starting Epoch 29 ------------------------------------------
Loss after 0/439 Batch is 0.90283203125 
Loss after 100/439 Batch is 0.79150390625 
Loss after 2

In [56]:
from datetime import datetime

format_data = "date_%d_%m_%y_time_%H_%M_%S"
timestamp = datetime.strftime(datetime.now(), format_data)
loss_name = "NST"


torch.save(
    Trainer.visual_student.state_dict(),
    f"results/{timestamp}_{loss_name}_CombinedVisual_DistilledModel.pt",
)
torch.save(
    Trainer.text_student.state_dict(),
    f"results/{timestamp}_{loss_name}_CombinedText_DistilledModel.pt",
)